In [2]:
import Pkg

pkgs = [
"DataFrames",
"StatsBase",
"StatsPlots",
"uCSV",
"ProgressMeter",
"Distances",
"Clustering",
"Colors",
"MultivariateStats"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [3]:
# kraken_db = "k2_pluspfp_20221209"
# kraken_db = "k2_pluspfp_08gb_20231009"
# kraken_db = "k2_pluspfp_16gb_20231009"
kraken_db = "k2_pluspfp_20231009"

"k2_pluspfp_20231009"

In [6]:
data_dir = joinpath(dirname(pwd()), "data")

results_dir = mkpath(joinpath(dirname(pwd()), "results"))

sample_directories = readdir(joinpath(data_dir, "samples"), join=true)

kraken_reports = String[]
for sample_directory in sample_directories
    trim_galore_directories = filter(x -> occursin(r"trimgalore$", x), readdir(sample_directory, join=true))
    for trim_galore_directory in trim_galore_directories
        # @show trim_galore_directory
        kraken_directories = filter(x -> occursin(r"kraken", x), readdir(trim_galore_directory, join=true))
        for kraken_directory in kraken_directories
            reports = filter(x -> occursin(r"kraken-report\.tsv$", x), readdir(kraken_directory, join=true))
            append!(kraken_reports, reports)
        end
    end
end
kraken_reports = filter(x -> occursin(kraken_db, x), kraken_reports)

51-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 305 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 313 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 313 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 313 bytes ⋯ "pfp_20231009.kraken-report.tsv"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 313 bytes ⋯ "pfp_20231009.kraken-r

In [7]:
taxon_levels = Mycelia.list_ranks()

9-element Vector{String}:
 "top"
 "superkingdom"
 "kingdom"
 "phylum"
 "class"
 "order"
 "family"
 "genus"
 "species"

In [184]:
# i = 1
# i = 2
# i = 3
# i = 4
# i = 5
# i = 6
# i = 7
# i = 8
i = 9

(taxon_index, taxon_level) = collect(enumerate(taxon_levels))[i]
println("$(taxon_index) - $(taxon_level)")
rank_table = Mycelia.list_rank(taxon_level)

9 - species


Row,taxid,name
,Int64,String
1,46014,Plasmavirus L2
2,174676,unidentified Fusellovirus
3,244589,Sulfolobus spindle-shaped virus 1
4,244590,Sulfolobus spindle-shaped virus 2
5,459290,Sulfolobus spindle-shaped virus 4
6,459291,Sulfolobus spindle-shaped virus 5
7,693628,Sulfolobus spindle-shaped virus 7
8,2032563,Sulfolobus spindle-shaped virus 8
9,2032573,Sulfolobus spindle-shaped virus 9


In [185]:
# HERE IS WHERE WE APPLY VIRAL FILTERING
# if taxon_index > 2
# viral_taxids = Set(Mycelia.list_subtaxa(10239))
#     taxid_is_viral = map(taxid -> taxid in viral_taxids, rank_table[!, "taxid"])
#     rank_table = rank_table[taxid_is_viral, :]
# end
# rank_table

In [186]:
# turn me into a function that accepts a rank table and a list of kraken reports as well as an output

rank_taxids = Set(rank_table[!, "taxid"])
cross_sample_taxon_report = joinpath(results_dir, "$(kraken_db).$(taxon_level).tsv")
# cross_sample_taxon_report_table = DataFrames.DataFrame()
# ProgressMeter.@showprogress for kraken_report in kraken_reports
#     report_table = Mycelia.read_kraken_report(kraken_report)
#     taxon_level_report = report_table[map(x -> x in rank_taxids, report_table[!, "ncbi_taxonid"]), :]
#     taxon_level_report[!, "sample_identifier"] .= basename(kraken_report)
#     append!(cross_sample_taxon_report_table, taxon_level_report)
# end
# uCSV.write(cross_sample_taxon_report, cross_sample_taxon_report_table, delim='\t')
cross_sample_taxon_report_table = DataFrames.DataFrame(uCSV.read(cross_sample_taxon_report, delim='\t', header=1))

Row,percentage_of_fragments_at_or_below_taxon,number_of_fragments_at_or_below_taxon,number_of_fragments_assigned_directly_to_taxon,rank,ncbi_taxonid,scientific_name,sample_identifier
,Float64,Int64,Int64,String,Int64,String,String
1,30.18,16102,16102,S,9606,Homo sapiens,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
2,0.01,5,0,S,1071379,Tetrapisispora blattae,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
3,0.0,0,0,S,113608,Tetrapisispora phaffii,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
4,0.0,2,2,S,28985,Kluyveromyces lactis,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
5,0.0,1,0,S,4911,Kluyveromyces marxianus,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
6,0.0,2,0,S,27288,Naumovozyma castellii,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
7,0.0,1,0,S,27289,Naumovozyma dairenensis,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
8,0.01,3,3,S,42260,Zygotorulaspora mrakii,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv
9,0.0,1,1,S,27291,Saccharomyces paradoxus,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv


In [187]:
# this could be a function here

In [188]:
cross_sample_taxon_report_summary = cross_sample_taxon_report_table[!, 
    DataFrames.Not([
            "percentage_of_fragments_at_or_below_taxon",
            "number_of_fragments_assigned_directly_to_taxon",
            "rank"
        ])]
cross_sample_taxon_report_summary[!, "taxon"] = map(row -> string(row["ncbi_taxonid"]) * "_" * row["scientific_name"], DataFrames.eachrow(cross_sample_taxon_report_summary))
cross_sample_taxon_report_summary = cross_sample_taxon_report_summary[!, DataFrames.Not([
            "ncbi_taxonid",
            "scientific_name"
        ])]

# assert sortedness & uniqueness (should be a no-op)
unique!(DataFrames.sort!(cross_sample_taxon_report_summary, ["sample_identifier", "taxon"]))

Row,number_of_fragments_at_or_below_taxon,sample_identifier,taxon
,Int64,String,String
1,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1000373_Rosellinia necatrix quadrivirus 1
2,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1001341_Vernonia yellow vein Fujian virus
3,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,100177_Borreliella lusitaniae
4,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,100217_Epizootic haematopoietic necrosis virus
5,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,100225_Austwickia chelonae
6,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1002672_Candidatus Pelagibacter sp. IMCC9063
7,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1002681_Sida golden mosaic Lara virus
8,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1002689_Occallatibacter riparius
9,0,LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,1002724_Tunavirus Shfl1


In [189]:
# taxa = String[]
# samples = String[]
# n_samples = length(unique(cross_sample_taxon_report_summary[!, "sample_identifier"]))
# n_taxa = length(unique(cross_sample_taxon_report_summary[!, "taxon"]))
# values = zeros(n_samples, n_taxa)

taxa = sort(unique(cross_sample_taxon_report_summary[!, "taxon"]))
samples = sort(unique(cross_sample_taxon_report_summary[!, "sample_identifier"]))
values = zeros(length(samples), length(taxa))
ProgressMeter.@showprogress for (column_index, taxon_table) in enumerate(DataFrames.groupby(cross_sample_taxon_report_summary, "taxon"))
    taxon = taxon_table[1, "taxon"]
    @assert taxa[column_index] == taxon
    for (row_index, sample_table) in enumerate(DataFrames.groupby(taxon_table, "sample_identifier"))
        @assert DataFrames.nrow(sample_table) == 1
        row = sample_table[1, :]
        sample = row["sample_identifier"]
        @assert samples[row_index] == sample
        values[row_index, column_index] = row["number_of_fragments_at_or_below_taxon"]
    end
end
values

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


51×25153 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0   3.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0   8.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0   3.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  15.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   2.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  11.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   3.0
 ⋮          

In [190]:
# sort taxa so largest single sample taxa is first
taxa_frequency_ordering = sortperm(maximum.(eachcol(values)))
values = values[:, taxa_frequency_ordering]
taxa = taxa[taxa_frequency_ordering]
# find taxa that have no representation, and filter them out
taxa_is_detected = [maximum(col) >= 3 for col in eachcol(values)]
values = values[:, taxa_is_detected]
taxa = taxa[taxa_is_detected]

596-element Vector{String}:
 "101571_Burkholderia ubonensis"
 "104270_Tenacibaculum ovolyticum"
 "1054033_Arcobacter venerupis"
 "106590_Cupriavidus necator"
 "1069201_Aspergillus luchuensis"
 "110321_Sinorhizobium medicae"
 "114525_Saccharomyces mikatae"
 "114885_Mycoplasmopsis maculosa"
 "1159556_Ustilaginoidea virens"
 "116188_Streptomyces coeruleorubidus"
 "1165_Anabaena cylindrica"
 "1177216_Candidatus Profftia tarda"
 "118060_Enterococcus rotai"
 ⋮
 "3469_Papaver somniferum"
 "33011_Cutibacterium granulosum"
 "61646_Lelliottia amnigena"
 "3911_Vicia villosa"
 "4232_Helianthus annuus"
 "4236_Lactuca sativa"
 "35269_Woodchuck hepatitis virus"
 "3888_Pisum sativum"
 "4072_Capsicum annuum"
 "3369_Cryptomeria japonica"
 "1747_Cutibacterium acnes"
 "9606_Homo sapiens"

In [191]:
colorscheme = Colors.distinguishable_colors(length(taxa), [Colors.RGB(1,1,1), Colors.RGB(0,0,0)], dropseed=true)
normalized_values = values ./ sum(values, dims=2)
samples = string.(first.(split.(samples, '.')))

51-element Vector{String}:
 "LO_FA_CKDN230005699-1A_H3TLGDSX7_L1"
 "LO_FA_CKDN230005699-1A_H5HNJDSX7_L2"
 "LO_FA_CKDN230005699-1A_H5HVHDSX7_L1"
 "LO_FA_CKDN230005699-1A_H5HYWDSX7_L2"
 "LO_MO_CKDN230005698-1A_H3TLGDSX7_L1"
 "LO_MO_CKDN230005698-1A_H5HVHDSX7_L1"
 "LO_MO_CKDN230005698-1A_H5M5FDSX7_L2"
 "LO_MO_CKDN230005698-1A_H5MWHDSX7_L1"
 "LO_PROB_CKDN230005697-1A_H3TJYDSX7_L3"
 "LO_PROB_CKDN230005697-1A_H5HNJDSX7_L2"
 "RA_PROB_CKDN230005700-1A_H3TLGDSX7_L1"
 "RA_PROB_CKDN230005700-1A_H5HNJDSX7_L2"
 "RI_1BRO_CKDN230005691-1A_H3TJYDSX7_L1"
 ⋮
 "RI_MaUn_CKDN230005694-1A_H5MWHDSX7_L1"
 "RI_PROB_CKDN230005690-1A_H3TJYDSX7_L1"
 "RI_PROB_CKDN230005690-1A_H5HNJDSX7_L2"
 "RI_PROB_CKDN230005690-1A_H5HVHDSX7_L1"
 "RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2"
 "RI_UC_CKDN230005693-1A_H3TJYDSX7_L2"
 "RI_UC_CKDN230005693-1A_H5HNJDSX7_L2"
 "RI_UC_CKDN230005693-1A_H5HVHDSX7_L1"
 "RI_UC_CKDN230005693-1A_H5HYWDSX7_L2"
 "VA_PROB_CKDN230005701-1A_H3TLGDSX7_L1"
 "VA_PROB_CKDN230005701-1A_H5HNJDSX7_L2"
 "VA_PROB_

In [192]:
# fit_pca = MultivariateStats.fit(MultivariateStats.PCA, normalized_values')

# fit_pca = MultivariateStats.fit(MultivariateStats.PCA, normalized_values')
# transformed_observations = MultivariateStats.transform(fit_pca, normalized_values')

# top_level_groups = unique(map(x -> join(split(x, '_')[1:2], '_'), samples))
# # Colors.RGB(0,0,0), Colors.RGB(0.78, 0.129, 0.867)
# colorscheme = Colors.distinguishable_colors(length(top_level_groups), [Colors.RGB(1,1,1), Colors.RGB(0,0,0)], dropseed=true)
# xs = [Float64[] for group in top_level_groups]
# ys = [Float64[] for group in top_level_groups]
# zs = [Float64[] for group in top_level_groups]

# raw_xs = transformed_observations[1, :]
# if size(transformed_observations, 1) >= 2
#     raw_ys = transformed_observations[2, :]
# else
#     raw_ys = zeros(length(raw_xs))
# end

# if size(transformed_observations, 1) >= 3
#     raw_zs = transformed_observations[3, :]
# else
#     raw_zs = zeros(length(raw_xs))
# end

# for (sample, x, y, z) in zip(samples, raw_xs, raw_ys, raw_zs)
#     # @show sample, x, y
#     sample_groups = findall(x -> occursin(x, sample), top_level_groups)
#     @assert length(sample_groups) == 1
#     sample_group = first(sample_groups)
#     push!(xs[sample_group], x)
#     push!(ys[sample_group], y)
#     push!(zs[sample_group], z)
# end

# plot = 
# StatsPlots.scatter(
#     xs,
#     ys,
#     # zs,
#     xlabel = "PC1",
#     ylabel = "PC2",
#     # zlabel = "PC3",
#     labels = hcat(top_level_groups...),
#     title = "read-classification - kraken - $(kraken_db) - $(taxon_level)",
#     titlefontsize = 10,
#     legend = :outertopright,
#     size = (640, 480),
#     margins = 20StatsPlots.px,
#     seriescolor = hcat(colorscheme...)
# )

# display(plot)
# for extension in [".png"]
#     file = joinpath(results_dir, "taxonomic-breakdowns.kraken.$(kraken_db).$(taxon_index).$(taxon_level).pca") * extension
#     StatsPlots.savefig(plot, file)
# end

In [193]:
# - loop through the above
# - make a matrix where each row is a taxon rank and each column is a dataset
# - make stacked barplots for each sample

In [194]:
colorscheme = Colors.distinguishable_colors(length(taxa), [Colors.RGB(1,1,1), Colors.RGB(0,0,0)], dropseed=true)
# top level classification rank to show absolute reads per sample
# not very helpful at lower ranks since low read depth samples are too hard to see breakdowns

bottommargin = (maximum(length.(samples)) * 3)
leftmargin = 150
rightmargin = 25
topmargin = 25

width = max(1920, (length(samples) * 10) + 300)
height = max(1080, bottommargin + 600)

if taxon_index in [1, 2]
    plot = StatsPlots.groupedbar(
        values,
        title = "read-classification - kraken - $(kraken_db) - $(taxon_level)",
        titlefontsize = 12,
        xticks = (1:length(samples), samples),
        xlims = (0, length(samples)+1),
        xtickfontsize = 6,
        size= (width, height),
        xrotation=90,
        ylabel = "number of reads",
        labels = hcat(taxa...),
        leftmargin = (leftmargin)StatsPlots.Plots.PlotMeasures.px,
        topmargin = (topmargin)StatsPlots.Plots.PlotMeasures.px,
        rightmargin = (rightmargin)StatsPlots.Plots.PlotMeasures.px,
        bottommargin = (bottommargin)StatsPlots.Plots.PlotMeasures.px,
        legendmargins = 0,
        legend = :outertopright,
        legendfontsize = 6,
        bar_position = :stack,
        bar_width=0.7,
        seriescolor = hcat(reverse(colorscheme)...)
    )
    # display(plot)
    # for extension in [".png", ".svg"]
    for extension in [".png"]
        file = joinpath(results_dir, "taxonomic-breakdowns.kraken.$(kraken_db).$(taxon_index).$(taxon_level).total-reads") * extension
        StatsPlots.savefig(plot, file)
    end
end

################################################################################################

# re-tune height to be proportional to size of taxa list (tall legends)
height = max(height, length(taxa)*11)
plot = StatsPlots.groupedbar(
    normalized_values,
    title = "read-classification - kraken - $(kraken_db) - $(taxon_level)",
    titlefontsize = 12,
    xticks = (1:length(samples), samples),
    xlims = (0, length(samples)+1),
    xtickfontsize = 6,
    size= (width, height),
    xrotation=90,
    ylabel = "proportion of reads",
    labels = hcat(taxa...),
    leftmargin = (leftmargin)StatsPlots.Plots.PlotMeasures.px,
    topmargin = (topmargin)StatsPlots.Plots.PlotMeasures.px,
    rightmargin = (rightmargin)StatsPlots.Plots.PlotMeasures.px,
    bottommargin = (bottommargin)StatsPlots.Plots.PlotMeasures.px,
    legendmargins = 0,
    legend = :outertopright,
    bar_position = :stack,
    bar_width=0.7,
    seriescolor = hcat(reverse(colorscheme)...),
    legendfontsize = 6,
)

# for extension in [".png", ".svg"]
for extension in [".png"]
    file = joinpath(results_dir, "taxonomic-breakdowns.kraken.$(kraken_db).$(taxon_index).$(taxon_level).normalized-reads") * extension
    StatsPlots.savefig(plot, file)
end

################################################################################################
distance_matrix = Distances.pairwise(Distances.Euclidean(), normalized_values, dims=1)
clustering = Clustering.hclust(distance_matrix, branchorder=:optimal)
cluster_plot = StatsPlots.plot(
    clustering,
    xlims = (0, length(samples)+1),
    title = "read-classification - kraken - $(kraken_db) - $(taxon_level)",
    titlefontsize = 12,
    bottommargin = 0StatsPlots.Plots.PlotMeasures.px,
    leftmargin = (leftmargin)StatsPlots.Plots.PlotMeasures.px,
    rightmargin = (rightmargin)StatsPlots.Plots.PlotMeasures.px,
    topmargin = (topmargin)StatsPlots.Plots.PlotMeasures.px,
    xticks = false,
    yticks = false,
    yaxis = false,
    xaxis = false
)
taxonomy_plot = StatsPlots.groupedbar(
    normalized_values[clustering.order, :],
    bar_position = :stack,
    leftmargin = (leftmargin)StatsPlots.Plots.PlotMeasures.px,
    topmargin = 0StatsPlots.Plots.PlotMeasures.px,
    rightmargin = (rightmargin)StatsPlots.Plots.PlotMeasures.px,
    bottommargin = (bottommargin)StatsPlots.Plots.PlotMeasures.px,
    legendmargins = 0,
    xticks = (1:length(samples), samples[clustering.order]),
    xtickfontsize = 4,
    xrotation=90,
    xlims = (0, length(samples)+1),
    size= (width, height),
    ylabel = "proportion of reads",
    seriescolor = hcat(reverse(colorscheme)...),
    legend = false,
    labels = hcat(taxa...),
    legendfontsize = 6,
)

plot = StatsPlots.plot(
    cluster_plot,
    taxonomy_plot,
    margins = 0StatsPlots.Plots.PlotMeasures.px,
    layout=StatsPlots.grid(2,1, heights=[0.2,0.8])
)
# display(plot)
# for extension in [".png", ".svg"]
for extension in [".png"]
    file = joinpath(results_dir, "taxonomic-breakdowns.kraken.$(kraken_db).$(taxon_index).$(taxon_level).normalized-reads.clustered") * extension
    StatsPlots.savefig(plot, file)
end